In [74]:
import os
from ranger import Ranger

import torch
from torch import nn
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.nn.utils.prune as prune

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import Normalize
from torchmetrics import Accuracy

import torch.optim as optim
from cleverhans.torch.attacks.projected_gradient_descent import (projected_gradient_descent)

import quantus
import captum
from captum.attr import Saliency, IntegratedGradients, NoiseTunnel

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
import copy
import gc

import warnings
warnings.filterwarnings('ignore')

In [76]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [80]:
batch_size = 64
epochs = 50
train_path = 'datasets/imagenette2/train'
val_path =  'datasets/imagenette2/val'
train_dataloader = torch.utils.data.DataLoader(datasets.ImageFolder(train_path, 
                                                                   transform = transforms.Compose([
                                                                        transforms.RandomResizedCrop(224),
                                                                        transforms.RandomHorizontalFlip(),
                                                                        transforms.ToTensor(),
                                                                        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                             std=[0.229, 0.224, 0.225])
                                                                   ])), batch_size = batch_size, shuffle=True)

test_dataloader = torch.utils.data.DataLoader(datasets.ImageFolder(val_path,
                                                               transform=transforms.Compose([
                                                                   transforms.ToTensor(),
                                                                   transforms.Resize([224, 224]),
                                                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                        std=[0.229, 0.224, 0.225])
                                                               ])),batch_size=batch_size, shuffle=True)

In [82]:
classes = ('tench', 'springer', 'casette_player', 'chain_saw','church', 'French_horn', 'garbage_truck', 'gas_pump', 'golf_ball', 'parachute')

In [84]:
%run models.ipynb
%run utils.ipynb
%run metrics.ipynb

In [86]:
from pathlib import Path

model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_imagenette.pth"
model_save_path = model_path / model_name
model_normal = resnet_18(filter='None', filter_layer=0)
model_normal.load_state_dict(torch.load(model_save_path))

<All keys matched successfully>

In [88]:
# Model to GPU and eval mode.
model_normal.to(device)
model_normal.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_normal, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Test accuracy for Resnet 18 Normal is: {(100 * test_acc):.2f}%")

Test accuracy for Resnet 18 Normal is: 81.43%


# Unstructured Post Pruning

In [90]:
from pathlib import Path

model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_imagenette.pth"
model_save_path = model_path / model_name
model_notune = resnet_18(filter='None', filter_layer=0)
model_notune.load_state_dict(torch.load(model_save_path))

<All keys matched successfully>

In [92]:
print(f"RESNET-18 global sparsity = {compute_sparsity_resnet(model_notune):.2f}%")

RESNET-18 global sparsity = 0.00%


In [93]:
import torch.nn.utils.prune as prune
for name, module in model_notune.named_modules():
    # prune 20% of weights/connections in for all hidden layaers-
    if isinstance(module, torch.nn.Conv2d):
        prune.l1_unstructured(module = module, name = 'weight', amount = 0.2)
    
    # prune 10% of weights/connections for output layer-
    elif isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module = module, name = 'weight', amount = 0.1)

In [98]:
print(f"RESNET-18 global sparsity = {compute_sparsity_resnet(model_notune):.2f}%")

RESNET-18 global sparsity = 19.99%


In [100]:
# Model to GPU and eval mode.
model_notune.to(device)
model_notune.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_notune, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Test accuracy for Resnet 18 pretrained no tuning Normal is: {(100 * test_acc):.2f}%")

Test accuracy for Resnet 18 pretrained no tuning Normal is: 77.91%


In [102]:
from pathlib import Path

model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_imagenette_no_tuned_l1.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_notune.state_dict(), f=model_save_path)

Saving the model: models\resnet_imagenette_no_tuned_l1.pth


In [104]:
model = model_notune
learning_rate = 1e-04
criterion = nn.CrossEntropyLoss(reduction="mean").cuda()
optimizer = Ranger(model.parameters(), lr = learning_rate, eps = 1e-06)

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers


In [106]:
def train_model(model, epochs):
    model.train()
    for epoch in range(epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            output = model(x_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()

        # Evaluate model!
        if epochs%10==0:
            predictions, labels = evaluate_model(model, test_dataloader, device)
            test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
            print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [108]:
model_tuned = train_model(model = model.to(device), epochs = epochs)

Epoch 1/50 - test accuracy: 80.36% and CE loss 0.64
Epoch 2/50 - test accuracy: 82.55% and CE loss 0.50
Epoch 3/50 - test accuracy: 82.80% and CE loss 0.25
Epoch 4/50 - test accuracy: 82.24% and CE loss 0.37
Epoch 5/50 - test accuracy: 83.11% and CE loss 0.29
Epoch 6/50 - test accuracy: 82.70% and CE loss 0.26
Epoch 7/50 - test accuracy: 82.73% and CE loss 0.18
Epoch 8/50 - test accuracy: 82.75% and CE loss 0.26
Epoch 9/50 - test accuracy: 82.98% and CE loss 0.39
Epoch 10/50 - test accuracy: 82.29% and CE loss 0.16
Epoch 11/50 - test accuracy: 83.29% and CE loss 0.26
Epoch 12/50 - test accuracy: 83.08% and CE loss 0.37
Epoch 13/50 - test accuracy: 81.63% and CE loss 0.20
Epoch 14/50 - test accuracy: 82.19% and CE loss 0.24
Epoch 15/50 - test accuracy: 82.93% and CE loss 0.21
Epoch 16/50 - test accuracy: 83.26% and CE loss 0.25
Epoch 17/50 - test accuracy: 82.42% and CE loss 0.19
Epoch 18/50 - test accuracy: 83.85% and CE loss 0.20
Epoch 19/50 - test accuracy: 82.17% and CE loss 0.11
Ep

In [109]:
# Model to GPU and eval mode.
model_tuned.to(device)
model_tuned.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_tuned, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 83.08%


In [110]:
from pathlib import Path

model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_imagenette_post_tuned_l1.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_tuned.state_dict(), f=model_save_path)

Saving the model: models\resnet_imagenette_post_tuned_l1.pth


# Global Post Train Prune

In [131]:
from pathlib import Path

model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_imagenette.pth"
model_save_path = model_path / model_name
model_notune = resnet_18(filter='None', filter_layer=0)
model_notune.load_state_dict(torch.load(model_save_path))

<All keys matched successfully>

In [133]:
print(f"RESNET-18 global sparsity = {compute_sparsity_resnet(model_notune):.2f}%")

RESNET-18 global sparsity = 0.00%


In [135]:
parameters_to_prune = (
    (model_notune.conv1, 'weight'),
    (model_notune.bn1, 'weight'),
    (model_notune.layer1[0].conv1, 'weight'),
    (model_notune.layer1[0].bn1, 'weight'),
    (model_notune.layer1[0].conv2, 'weight'),
    (model_notune.layer1[0].bn2, 'weight'),
    (model_notune.layer1[1].conv1, 'weight'),
    (model_notune.layer1[1].bn1, 'weight'),
    (model_notune.layer1[1].conv2, 'weight'),
    (model_notune.layer1[1].bn2, 'weight'),
    (model_notune.layer2[0].conv1, 'weight'),
    (model_notune.layer2[0].bn1, 'weight'),
    (model_notune.layer2[0].conv2, 'weight'),
    (model_notune.layer2[0].bn2, 'weight'),
    (model_notune.layer2[1].conv1, 'weight'),
    (model_notune.layer2[1].bn1, 'weight'),
    (model_notune.layer2[1].conv2, 'weight'),
    (model_notune.layer2[1].bn2, 'weight'),
    (model_notune.layer3[0].conv1, 'weight'),
    (model_notune.layer3[0].bn1, 'weight'),
    (model_notune.layer3[0].conv2, 'weight'),
    (model_notune.layer3[0].bn2, 'weight'),
    (model_notune.layer3[1].conv1, 'weight'),
    (model_notune.layer3[1].bn1, 'weight'),
    (model_notune.layer3[1].conv2, 'weight'),
    (model_notune.layer3[1].bn2, 'weight'),
    (model_notune.layer4[0].conv1, 'weight'),
    (model_notune.layer4[0].bn1, 'weight'),
    (model_notune.layer4[0].conv2, 'weight'),
    (model_notune.layer4[0].bn2, 'weight'),
    (model_notune.layer4[1].conv1, 'weight'),
    (model_notune.layer4[1].bn1, 'weight'),
    (model_notune.layer4[1].conv2, 'weight'),
    (model_notune.layer4[1].bn2, 'weight'),
    (model_notune.fc, 'weight')
)

prune_rates_global = [0.2, 0.3, 0.4, 0.5, 0.6]

In [137]:
for iter_prune_round in range(1):
    print(f"\n\nIterative Global pruning round = {iter_prune_round + 1}")
    
    # Prune layer-wise in a structured manner-
    prune.global_unstructured(
        parameters_to_prune,
        pruning_method = prune.L1Unstructured,
        amount = prune_rates_global[iter_prune_round]
        
    )

    # Print current global sparsity level-
    print(f"RESNET-18 global sparsity = {compute_sparsity_resnet(model_notune):.2f}%")



Iterative Global pruning round = 1
RESNET-18 global sparsity = 20.00%


In [139]:
# Model to GPU and eval mode.
model_notune.to(device)
model_notune.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_notune, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Test accuracy for Resnet 18 pretrained no tuning Normal is: {(100 * test_acc):.2f}%")

Test accuracy for Resnet 18 pretrained no tuning Normal is: 81.07%


In [141]:
from pathlib import Path

model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_imagenette_no_tuned_global.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_notune.state_dict(), f=model_save_path)

Saving the model: models\resnet_imagenette_no_tuned_global.pth


In [143]:
model = model_notune
learning_rate = 1e-03
criterion = nn.CrossEntropyLoss(reduction="mean").cuda()
optimizer = Ranger(model.parameters(), lr = learning_rate, eps = 1e-06)

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers


In [145]:
model_tuned = train_model(model = model.to(device), epochs = epochs)

Epoch 1/50 - test accuracy: 79.08% and CE loss 0.72
Epoch 2/50 - test accuracy: 78.27% and CE loss 0.53
Epoch 3/50 - test accuracy: 80.31% and CE loss 0.39
Epoch 4/50 - test accuracy: 76.61% and CE loss 0.74
Epoch 5/50 - test accuracy: 78.24% and CE loss 0.55
Epoch 6/50 - test accuracy: 81.53% and CE loss 0.58
Epoch 7/50 - test accuracy: 78.24% and CE loss 0.48
Epoch 8/50 - test accuracy: 79.85% and CE loss 0.59
Epoch 9/50 - test accuracy: 82.75% and CE loss 0.46
Epoch 10/50 - test accuracy: 79.36% and CE loss 0.71
Epoch 11/50 - test accuracy: 79.39% and CE loss 0.63
Epoch 12/50 - test accuracy: 82.32% and CE loss 0.65
Epoch 13/50 - test accuracy: 81.73% and CE loss 0.79
Epoch 14/50 - test accuracy: 83.62% and CE loss 0.39
Epoch 15/50 - test accuracy: 83.36% and CE loss 0.52
Epoch 16/50 - test accuracy: 82.68% and CE loss 0.44
Epoch 17/50 - test accuracy: 82.57% and CE loss 0.55
Epoch 18/50 - test accuracy: 84.03% and CE loss 0.69
Epoch 19/50 - test accuracy: 81.20% and CE loss 0.43
Ep

In [146]:
# Model to GPU and eval mode.
model_tuned.to(device)
model_tuned.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_tuned, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 86.80%


In [147]:
from pathlib import Path

model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_imagenette_post_tuned_global.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_tuned.state_dict(), f=model_save_path)

Saving the model: models\resnet_imagenette_post_tuned_global.pth


# Layered post Train Pruning

In [148]:
from pathlib import Path

model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_imagenette.pth"
model_save_path = model_path / model_name
model_notune = resnet_18(filter='None', filter_layer=0)
model_notune.load_state_dict(torch.load(model_save_path))

<All keys matched successfully>

In [149]:
print(f"RESNET-18 global sparsity = {compute_sparsity_resnet(model_notune):.2f}%")

RESNET-18 global sparsity = 0.00%


In [150]:
 # Prune layer-wise in a structured manner-
prune.ln_structured(model_notune.conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer1[0].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer1[0].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer1[0].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer1[0].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer1[1].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer1[1].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer1[1].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer1[1].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer2[0].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer2[0].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer2[0].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer2[0].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer2[1].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer2[1].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer2[1].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer2[1].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer3[0].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer3[0].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer3[0].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer3[0].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer3[1].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer3[1].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer3[1].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer3[1].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer4[0].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer4[0].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer4[0].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer4[0].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer4[1].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer4[1].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer4[1].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer4[1].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)

prune.ln_structured(model_notune.fc, name = "weight", amount = 0.1, n = 2, dim = 0)

# Print current global sparsity level-
print(f"Resnet structrured sparsity = {compute_sparsity_resnet(model_notune):.2f}%")

Resnet structrured sparsity = 9.99%


In [151]:
# Model to GPU and eval mode.
model_notune.to(device)
model_notune.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_notune, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Test accuracy for Resnet 18 pretrained no tuning Normal is: {(100 * test_acc):.2f}%")

Test accuracy for Resnet 18 pretrained no tuning Normal is: 20.00%


In [152]:
from pathlib import Path

model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_imagenette_no_tuned_layered.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_notune.state_dict(), f=model_save_path)

Saving the model: models\resnet_imagenette_no_tuned_layered.pth


In [153]:
model = model_notune
learning_rate = 1e-04
criterion = nn.CrossEntropyLoss(reduction="mean").cuda()
optimizer = Ranger(model.parameters(), lr = learning_rate, eps = 1e-06)

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers


In [154]:
model_tuned = train_model(model = model.to(device), epochs = epochs)

Epoch 1/50 - test accuracy: 69.99% and CE loss 1.12
Epoch 2/50 - test accuracy: 78.65% and CE loss 1.05
Epoch 3/50 - test accuracy: 81.25% and CE loss 0.77
Epoch 4/50 - test accuracy: 80.94% and CE loss 0.33
Epoch 5/50 - test accuracy: 81.68% and CE loss 0.43
Epoch 6/50 - test accuracy: 81.86% and CE loss 0.42
Epoch 7/50 - test accuracy: 81.10% and CE loss 0.44
Epoch 8/50 - test accuracy: 81.66% and CE loss 0.31
Epoch 9/50 - test accuracy: 82.55% and CE loss 0.28
Epoch 10/50 - test accuracy: 82.17% and CE loss 0.24
Epoch 11/50 - test accuracy: 82.80% and CE loss 0.33
Epoch 12/50 - test accuracy: 81.99% and CE loss 0.40
Epoch 13/50 - test accuracy: 82.75% and CE loss 0.27
Epoch 14/50 - test accuracy: 81.55% and CE loss 0.39
Epoch 15/50 - test accuracy: 82.45% and CE loss 0.39
Epoch 16/50 - test accuracy: 79.95% and CE loss 0.36
Epoch 17/50 - test accuracy: 82.39% and CE loss 0.33
Epoch 18/50 - test accuracy: 81.89% and CE loss 0.40
Epoch 19/50 - test accuracy: 82.34% and CE loss 0.39
Ep

In [155]:
# Model to GPU and eval mode.
model_tuned.to(device)
model_tuned.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_tuned, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 83.72%


In [156]:
from pathlib import Path

model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_imagenette_post_tuned_layered.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_tuned.state_dict(), f=model_save_path)

Saving the model: models\resnet_imagenette_post_tuned_layered.pth
